In [18]:
import os, sys, json
from pathlib import Path
import pandas as pd
import tqdm
import numpy as np

import librosa

import IPython.display as ipd

In [2]:
se_root_path = '/scratch/work/marl/datasets/sound_datasets/fsd50k/'
_se_root_path = Path(se_root_path)

In [3]:
with open('source_datasets/ontology.json') as f:
    ontology = json.load(f)
# ontology_str = json.dumps(ontology, indent=4)
ontology_dict = {item['id']: item for item in ontology}

In [4]:
classwise_file_list = []
db_path = Path('/scratch/pk3251/appdir/Github/SpatialScaper/demo_notebooks/source_datasets/single_source_samples/FSD50K')
classwise_file_list += sorted(db_path.glob('*.tsv'))

In [5]:
# classwise_file_list = classwise_file_list[:3]
# classwise_file_list

In [9]:
_classes = set()
_nb_classes = 0
_min_samples_per_class = 30


file_dict = {}
for cls_ix, cls in enumerate(classwise_file_list):
    try:
        _cls_file_list = pd.read_csv(cls, sep='\t', header=None).values
        _cls_file_list = tqdm.tqdm(_cls_file_list, desc=f'Loading {cls.stem} samples, {cls_ix+1}/{len(classwise_file_list)}...')
    except:
        print('exception', cls)
        continue
    _filtered_cls_file_list = []
    if 'FSD50K' in str(cls):
        print(cls)
        for _item in _cls_file_list:
            fname, duration, mids, split = _item

            audiofile = str(fname) + '.wav'
            if split == 'train':
                se_path = _se_root_path / 'FSD50K.dev_audio'
            elif split == 'test':
                se_path = _se_root_path / 'FSD50K.eval_audio'
            audiofile = str(se_path/audiofile)
            duration = librosa.get_duration(path=audiofile)
            # print(audiofile, duration)

            if duration <= 10 and duration >= 0.3:
                _filtered_cls_file_list.append([audiofile, [0, duration], mids])
            elif duration > 10:
                audio, sr = librosa.load(audiofile, sr=None)
                audio, onoffset = librosa.effects.trim(audio, top_db=30)
                onoffset = onoffset / sr
                duration = onoffset[1] - onoffset[0]
                if duration <= 10:
                    _filtered_cls_file_list.append(
                        [audiofile, onoffset, mids])
                elif duration > 10 and duration <= 10.5:
                    _filtered_cls_file_list.append(
                        [audiofile, [onoffset[0], onoffset[0]+10], mids])
                else:
                    nSegments = np.ceil(duration/10).astype(int)
                    for idx in range(nSegments):
                        if idx == nSegments-1:
                            _filtered_cls_file_list.append(
                                [audiofile, [onoffset[0]+idx*10, onoffset[1]], mids])
                        else:
                            _filtered_cls_file_list.append(
                                [audiofile, [onoffset[0]+idx*10, onoffset[0]+(idx+1)*10], mids])
            
    if cls.stem not in file_dict.keys():
        file_dict[cls.stem] = []
    file_dict[cls.stem] += _filtered_cls_file_list
filtered_file_dict = {}
for cls in file_dict.keys():      
    if len(file_dict[cls]) >= _min_samples_per_class:
        filtered_file_dict[cls] = file_dict[cls]
        _classes.add(cls)
    else:
        print('Class {} has {} samples, which is less than the minimum number of samples per class ({}).'.format(
            cls, len(file_dict[cls]), _min_samples_per_class))  

_nb_classes = len(_classes)
_classes = sorted(list(_classes))
print(_nb_classes, _classes)
# return filtered_file_dict

exception /scratch/pk3251/appdir/Github/SpatialScaper/demo_notebooks/source_datasets/single_source_samples/FSD50K/_m_01280g.tsv


Loading _m_012f08 samples, 2/199...:   0%|                                                                                                                                                                                                             | 0/65 [00:00<?, ?it/s]

/scratch/pk3251/appdir/Github/SpatialScaper/demo_notebooks/source_datasets/single_source_samples/FSD50K/_m_012f08.tsv


Loading _m_0130jx samples, 3/199...:   5%|█████████▉                                                                                                                                                                                           | 5/99 [00:00<00:02, 43.43it/s]

/scratch/pk3251/appdir/Github/SpatialScaper/demo_notebooks/source_datasets/single_source_samples/FSD50K/_m_0130jx.tsv


...

/scratch/pk3251/appdir/Github/SpatialScaper/demo_notebooks/source_datasets/single_source_samples/FSD50K/_t_dd00130.tsv


Loading _t_dd00134 samples, 199/199...:   7%|█████████████▌                                                                                                                                                                                    | 4/57 [00:00<00:02, 25.82it/s]

/scratch/pk3251/appdir/Github/SpatialScaper/demo_notebooks/source_datasets/single_source_samples/FSD50K/_t_dd00134.tsv


Loading _t_dd00134 samples, 199/199...: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:02<00:00, 27.74it/s]

Class _m_01qbl has 7 samples, which is less than the minimum number of samples per class (30).
Class _m_07r10fb has 26 samples, which is less than the minimum number of samples per class (30).
Class _m_07rkbfh has 21 samples, which is less than the minimum number of samples per class (30).
Class _m_0838f has 18 samples, which is less than the minimum number of samples per class (30).
Class _m_0_ksk has 2 samples, which is less than the minimum number of samples per class (30).
Class _m_0fx80y has 7 samples, which is less than the minimum number of samples per class (30).
Class _m_0j45pbj has 20 samples, which is less than the minimum number of samples per class (30).
Class _m_0jb2l has 6 samples, which is less than the minimum number of samples per class (30).
Class _m_0l14md has 22 samples, which is less than the minimum number of samples per class (30).
180 ['_m_012f08', '_m_0130jx', '_m_013y1f', '_m_014zdl', '_m_015lz1', '_m_015p6', '_m_018vs', '_m_0195fx', '_m_0199g', '_m_019jd', '

In [10]:
print(len(file_dict))

189


In [11]:
dataset_dir = '/home/pk3251/vast/DATA/spatial_eval_metrics/dataset/curated_source_files'

for class_id in file_dict.keys():
    #print(class_id, '|', ontology_dict[class_id.replace('_','/', 2)]['name'], '|', len(file_dict[class_id]))
    # os.makedirs(Path(dataset_dir) / class_id)
    print(file_dict[class_id])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [12]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        return json.JSONEncoder.default(self, obj)

with open("file_dict.json", "w") as f:
    _ = json.dumps(file_dict, cls=NumpyEncoder)
    json.dump(_, f)

In [13]:
with open("file_dict.json", "r") as f:
    print(f)
    # _ = json.load(f)
    # _ = json.loads(_)

<_io.TextIOWrapper name='file_dict.json' mode='r' encoding='utf-8'>


In [14]:
_['_m_012f08']

TypeError: string indices must be integers, not 'str'

In [39]:
aud1, _ = librosa.load('/home/pk3251/vast/DATA/spatial_eval_metrics/dataset/curated_single_source_stems/_t_dd00134/245359_51.wav', sr=44100)
ipd.Audio(aud1, rate=44100)

In [46]:
rooms = ['bomb_shelter', 'gym:', 'pb132', 'pc226', 'sa203', 'sc203', 'se203', 'tb103', 'tc352']
[rooms[a] for a in np.random.randint(0, len(rooms), 10)]

['sa203',
 'tb103',
 'gym:',
 'gym:',
 'bomb_shelter',
 'se203',
 'pc226',
 'sa203',
 'tb103',
 'tc352']

In [53]:
classwise_file_list = []
db_path = Path('/scratch/pk3251/appdir/Github/SpatialScaper/demo_notebooks/source_datasets/single_source_samples/FSD50K')
classwise_file_list += sorted(db_path.glob('*.tsv'))
for cls_ix, cls in enumerate(classwise_file_list):
    try:
        print('Running Class', cls)
        _cls_file_list = pd.read_csv(cls, sep='\t', header=None).values
    except:
        print('exception', cls)
        continue
    _filtered_cls_file_list = []
    if 'FSD50K' in str(cls):
        for _item in _cls_file_list:
            fname, duration, mids, split = _item

            audiofile = str(fname) + '.wav'
            if split == 'train':
                se_path = Path(se_root_path) / 'FSD50K.dev_audio'
            elif split == 'test':
                se_path = Path(se_root_path) / 'FSD50K.eval_audio'
            audiofile = str(se_path/audiofile)
            duration = librosa.get_duration(path=audiofile)
            print(audiofile, duration)

Running Class /scratch/pk3251/appdir/Github/SpatialScaper/demo_notebooks/source_datasets/single_source_samples/FSD50K/_m_01280g.tsv
exception /scratch/pk3251/appdir/Github/SpatialScaper/demo_notebooks/source_datasets/single_source_samples/FSD50K/_m_01280g.tsv
Running Class /scratch/pk3251/appdir/Github/SpatialScaper/demo_notebooks/source_datasets/single_source_samples/FSD50K/_m_012f08.tsv
/scratch/work/marl/datasets/sound_datasets/fsd50k/FSD50K.dev_audio/154627.wav 15.562517006802722
/scratch/work/marl/datasets/sound_datasets/fsd50k/FSD50K.dev_audio/154628.wav 24.56251700680272
/scratch/work/marl/datasets/sound_datasets/fsd50k/FSD50K.dev_audio/349188.wav 24.34768707482993
/scratch/work/marl/datasets/sound_datasets/fsd50k/FSD50K.dev_audio/76817.wav 14.863673469387756
/scratch/work/marl/datasets/sound_datasets/fsd50k/FSD50K.dev_audio/262680.wav 16.996666666666666
/scratch/work/marl/datasets/sound_datasets/fsd50k/FSD50K.dev_audio/121886.wav 16.770249433106574
/scratch/work/marl/datasets/s

KeyboardInterrupt: 